In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:

spark = SparkSession.builder \
    .appName("MemoryTuning") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/28 20:10:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet

--2025-02-28 20:06:36--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.126, 54.230.209.72, 54.230.209.200, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 472757547 (451M) [binary/octet-stream]
Saving to: ‘fhvhv_tripdata_2024-01.parquet.1’

fhvhv_tripdata_2024 100%[===================>] 450.86M  84.0MB/s    in 5.3s    

2025-02-28 20:06:41 (84.4 MB/s) - ‘fhvhv_tripdata_2024-01.parquet.1’ saved [472757547/472757547]



In [3]:
df = spark.read \
     .option("header","true") \
     .parquet("fhvhv_tripdata_2024-01.parquet")

In [4]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2024, 1, 1, 0, 21, 47), on_scene_datetime=datetime.datetime(2024, 1, 1, 0, 25, 6), pickup_datetime=datetime.datetime(2024, 1, 1, 0, 28, 8), dropoff_datetime=datetime.datetime(2024, 1, 1, 1, 5, 39), PULocationID=161, DOLocationID=158, trip_miles=2.83, trip_time=2251, base_passenger_fare=45.61, tolls=0.0, bcf=1.25, sales_tax=4.05, congestion_surcharge=2.75, airport_fee=0.0, tips=0.0, driver_pay=40.18, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag='N', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2024, 1, 1, 0, 10, 56), on_scene_datetime=datetime.datetime(2024, 1, 1, 0, 11, 8), pickup_datetime=datetime.datetime(2024, 1, 1, 0, 12, 53), dropoff_datetime=datetime.datetime(2024, 1, 1, 0, 20, 5), PULocationID=137, DOLoc

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('sha

In [6]:
df = df.repartition(4)

In [7]:
df.write.parquet('fhvhv/2024/01/')

In [8]:
df = spark.read.parquet('fhvhv/2024/01/')

In [9]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

In [17]:
## Select columns
## Filter by column values
df.select(['pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID']).filter(df.hvfhs_license_num == 'HV0003').show() 

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2024-01-01 23:03:53|2024-01-01 23:13:39|          39|          72|
|2024-01-02 17:03:13|2024-01-02 17:06:52|         208|         208|
|2024-01-03 12:33:35|2024-01-03 12:52:18|         133|          14|
|2024-01-02 15:01:10|2024-01-02 15:13:12|           9|           9|
|2024-01-02 18:07:39|2024-01-02 18:15:07|         156|          23|
|2024-01-03 14:23:41|2024-01-03 14:38:37|         107|         261|
|2024-01-01 01:19:53|2024-01-01 01:32:42|         197|         258|
|2024-01-02 21:27:32|2024-01-02 21:33:38|         143|         239|
|2024-01-04 07:20:50|2024-01-04 07:26:31|          90|          68|
|2024-01-01 07:42:18|2024-01-01 08:05:21|          36|         249|
|2024-01-04 15:59:18|2024-01-04 16:05:05|         112|         112|
|2024-01-02 07:56:49|2024-01-02 08:03:47|       

In [18]:
from pyspark.sql import functions as F

In [21]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select(['pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID']) \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2024-01-01|  2024-01-01|          39|          72|
| 2024-01-02|  2024-01-02|         208|         208|
| 2024-01-03|  2024-01-03|         133|          14|
| 2024-01-02|  2024-01-02|           9|           9|
| 2024-01-02|  2024-01-02|         156|          23|
| 2024-01-03|  2024-01-03|         107|         261|
| 2024-01-01|  2024-01-01|         197|         258|
| 2024-01-02|  2024-01-02|         143|         239|
| 2024-01-01|  2024-01-01|         162|         196|
| 2024-01-01|  2024-01-01|          25|         256|
| 2024-01-04|  2024-01-04|          90|          68|
| 2024-01-04|  2024-01-04|          42|          74|
| 2024-01-01|  2024-01-01|          36|         249|
| 2024-01-04|  2024-01-04|         112|         112|
| 2024-01-02|  2024-01-02|         152|          41|
| 2024-01-01|  2024-01-01|          39|       

In [23]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's{num:03x}'
    else:
        return f'e{num:03x}'        